In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# 1. ПОДГОТОВКА ДАННЫХ
# Загружаем данные (используем ваш текст из сообщения)
df = pd.read_csv('Cervical_cancer_Risk_factors.csv', na_values='?')

# Удаляем лишние колонки и заполняем пропуски медианой
df = df.drop(['STDs: Time since first diagnosis', 'STDs: Time since last diagnosis'], axis=1)
imputer = SimpleImputer(strategy='median')
df_clean = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Целевая переменная - Biopsy (результат биопсии)
X = df_clean.drop(['Hinselmann', 'Schiller', 'Citology', 'Biopsy'], axis=1)
y = df_clean['Biopsy']

# Обучаем модель
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X, y)

def predict_cancer_probability():
    print("\n--- Введите данные пациента для прогноза ---")
    
    try:
        # Мы создадим пустой вектор со всеми признаками (заполненный медианами)
        # и будем менять в нем только те значения, которые введет пользователь
        user_data = df_clean.drop(['Hinselmann', 'Schiller', 'Citology', 'Biopsy'], axis=1).median().values.reshape(1, -1)
        feature_names = X.columns.tolist()

        # Запрашиваем ключевые значения
        age = float(input("Возраст: "))
        partners = float(input("Количество половых партнеров: "))
        first_intercourse = float(input("Возраст первого полового акта: "))
        pregnancies = float(input("Количество беременностей: "))
        smokes_yrs = float(input("Стаж курения (лет, если нет - 0): "))
        hormonal_yrs = float(input("Стаж приема гормональных контрацептивов (лет, если нет - 0): "))
        iud_yrs = float(input("Сколько лет стоит внутриматочная спираль (если нет - 0): "))
        stds = float(input("Были ли СТД/ЗППП (1 - да, 0 - нет): "))

        # Записываем ввод в нужные позиции вектора
        inputs = {
            'Age': age,
            'Number of sexual partners': partners,
            'First sexual intercourse': first_intercourse,
            'Num of pregnancies': pregnancies,
            'Smokes (years)': smokes_yrs,
            'Hormonal Contraceptives (years)': hormonal_yrs,
            'IUD (years)': iud_yrs,
            'STDs': stds
        }

        for name, value in inputs.items():
            if name in feature_names:
                idx = feature_names.index(name)
                user_data[0][idx] = value

        # Предсказание вероятности
        probability = model.predict_proba(user_data)[0][1]
        
        print("\n" + "="*30)
        print(f"РЕЗУЛЬТАТ АНАЛИЗА:")
        print(f"Вероятность патологии по результатам биопсии: {probability:.1%}")
        
        if probability > 0.5:
            print("РЕКОМЕНДАЦИЯ: Высокий риск. Необходима консультация специалиста.")
        elif probability > 0.2:
            print("РЕКОМЕНДАЦИЯ: Средний риск. Рекомендуется плановое обследование.")
        else:
            print("РЕКОМЕНДАЦИЯ: Низкий риск.")
        print("="*30)

    except ValueError:
        print("Ошибка: Пожалуйста, вводите только числа.")

# Запуск функции предсказания
predict_cancer_probability()


--- Введите данные пациента для прогноза ---

РЕЗУЛЬТАТ АНАЛИЗА:
Вероятность патологии по результатам биопсии: 3.0%
РЕКОМЕНДАЦИЯ: Низкий риск.


/opt/anaconda3/envs/cercival_cancer/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
